In [1]:
import tensorflow as tf

In [20]:
import math

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
X=mnist.train.images
y=mnist.train.labels

In [4]:
mnist.test.images.shape

(10000, 784)

In [5]:
print (X.shape)
print (y.shape)

(55000, 784)
(55000, 10)


In [6]:
X = tf.placeholder("float", [None,784])
Y_=tf.placeholder("float",[None,10])

In [7]:
# five layers and their number of neurons (tha last layer has 10 softmax neurons)
L = 200
M = 100
N = 60
O = 30

In [8]:
# Weights initialised with small random values between -0.2 and +0.2
# When using RELUs, make sure biases are initialised with small *positive* values for example 0.1 = tf.ones([K])/10
W1 = tf.Variable(tf.truncated_normal([784, L], stddev=0.1))  # 784 = 28 * 28
B1 = tf.Variable(tf.zeros([L]))

W2 = tf.Variable(tf.truncated_normal([L, M], stddev=0.1))
B2 = tf.Variable(tf.zeros([M]))

W3 = tf.Variable(tf.truncated_normal([M,N], stddev=0.1))
B3 = tf.Variable(tf.zeros([N]))

W4 = tf.Variable(tf.truncated_normal([N, O], stddev=0.1))
B4 = tf.Variable(tf.zeros([O]))

W5 = tf.Variable(tf.truncated_normal([O, 10], stddev=0.1))
B5 = tf.Variable(tf.zeros([10]))

In [9]:
# The model
Y1 = tf.nn.relu(tf.matmul(X, W1) + B1)
Y2 = tf.nn.relu(tf.matmul(Y1, W2) + B2)
Y3 = tf.nn.relu(tf.matmul(Y2, W3) + B3)
Y4 = tf.nn.relu(tf.matmul(Y3, W4) + B4)
Ylogits = tf.matmul(Y4, W5) + B5
Y = tf.nn.softmax(Ylogits)

In [11]:
training_epochs = 25
batch_size = 100

In [22]:
# accuracy of the trained model, between 0 (worst) and 1 (best)
correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [10]:
# cross-entropy loss function (= -sum(Y_i * log(Yi)) ), normalised for batches of 100  images
# TensorFlow provides the softmax_cross_entropy_with_logits function to avoid numerical stability
# problems with log(0) which is NaN
cost= tf.nn.softmax_cross_entropy_with_logits(logits=Ylogits, labels=Y_)
cost = tf.reduce_mean(cost)

In [13]:
lr = tf.placeholder(tf.float32)

In [14]:
optimizer= tf.train.AdamOptimizer(lr).minimize(cost)

In [15]:
init =tf.global_variables_initializer()

In [16]:
# learning rate decay
max_learning_rate = 0.003
min_learning_rate = 0.0001
decay_speed = 2000.0 # 0.003-0.0001-2000=>0.9826 done in 5000 iterations
#learning_rate = min_learning_rate + (max_learning_rate - min_learning_rate) * math.exp(-i/decay_speed)

In [23]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(training_epochs):
        total_batch = int(mnist.train.num_examples/batch_size)
        c=0.0
        avg_cost=0.0
        # Loop over all batches
        for i in range(total_batch):
            
            learning_rate = min_learning_rate + (max_learning_rate - min_learning_rate) * math.exp(-i/decay_speed)
            
            batch_X, batch_Y = mnist.train.next_batch(batch_size)
            #print(batch_X.shape)
            _,c=sess.run([optimizer,cost], feed_dict={X:batch_X,Y_:batch_Y,lr: learning_rate})
            avg_cost += c / total_batch

        print("epchos {} cross entropy={}".format(epoch+1,avg_cost))
            
    print ("Optimization Finished!")

    # Test model
    #correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy for 3000 examples
    #accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print( "Accuracy:", accuracy.eval({X: mnist.test.images, Y_: mnist.test.labels}))

epchos 1 cross entropy=0.2749935317513617
epchos 2 cross entropy=0.11780536869032825
epchos 3 cross entropy=0.08314971835098482
epchos 4 cross entropy=0.06318846847989006
epchos 5 cross entropy=0.05431218413001092
epchos 6 cross entropy=0.0448232469589195
epchos 7 cross entropy=0.04066229906060144
epchos 8 cross entropy=0.03306623438002797
epchos 9 cross entropy=0.03185356917050248
epchos 10 cross entropy=0.030637005179993463
epchos 11 cross entropy=0.025852860826258147
epchos 12 cross entropy=0.02586389745194043
epchos 13 cross entropy=0.024558496682544825
epchos 14 cross entropy=0.023593054039255646
epchos 15 cross entropy=0.018100853314337504
epchos 16 cross entropy=0.021212865092423833
epchos 17 cross entropy=0.018586516214284293
epchos 18 cross entropy=0.014012875405599258
epchos 19 cross entropy=0.019006634854952378
epchos 20 cross entropy=0.0203057218112173
epchos 21 cross entropy=0.013126200907764441
epchos 22 cross entropy=0.014855445007937796
epchos 23 cross entropy=0.0154359